In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import statsmodels.api as sm

In [68]:
os.chdir(r'C:\Users\mwendwa.kiko\Documents\Personal_Kiko\Old_UofT\Research\WSTLUR 2023\Results')

In [69]:
scenario_name = "After Eric's comments"

In [70]:
# Read in the params
with open(f'{scenario_name}_params.pkl', 'rb') as f:
    params = pd.read_pickle(f)

In [71]:
params

array([ 9.02204642e-02, -2.29457120e-01,  1.85613379e-01, -2.14432184e-02,
       -1.90755729e-01,  1.38010672e-01, -1.34455169e-01, -1.55763215e-03,
        5.36058719e-02, -2.88336119e-02,  2.11491033e-02, -4.88324758e-03,
        9.79185127e-02, -6.89082729e-02, -1.89875881e-02,  9.28276338e-01,
       -2.67352585e-01,  2.83488377e-01,  5.22204779e-02, -4.28867448e-01,
        1.90237085e-01, -3.51515710e-02, -1.19914188e-02,  2.50531240e+00,
        4.17254564e+00,  4.64432338e-02, -2.05873546e-01,  1.70906059e-01,
        4.44942434e-02, -2.37481579e-01, -8.66988566e-02, -3.03859552e-01,
       -3.36480772e-03,  1.23552538e-01,  1.27813838e-01, -2.74777554e-03,
        1.07876003e-04,  2.56495241e-01,  8.69150442e-03,  7.08831414e-02,
        9.84334126e-01, -2.24925889e-01,  4.10854193e-01, -4.40600704e-01,
       -2.19773138e-01,  8.60494572e-02, -3.00552689e-02, -3.55829514e-02,
        1.14710266e-01])

In [72]:
data = pd.read_pickle(r'C:\Users\mwendwa.kiko\Documents\Personal_Kiko\Old_UofT\Research\WSTLUR 2023\df_persons_wsltur_with_entropy_socio_eco_highway_exit_logsum.pkl')

In [73]:
# Data processing
data.dropna(subset=['Household_size', 'Number_cars'], inplace=True)
# Get dummies for column Q8
data = pd.get_dummies(data, prefix='Q8', drop_first=True, columns=['Q8'])
data[['Q8_Male', 'Q8_Other']].astype('int64')
data['Length_of_cycle_lanes_within_radius'].fillna(data['Length_of_cycle_lanes_within_radius'].median(), inplace=True)
data['Distance_to_closest_bus_station'].fillna(data['Distance_to_closest_bus_station'].median(), inplace=True)
data['Popn_density'].fillna(data['Popn_density'].median(), inplace=True)
data['Apartment percentage'].fillna(data['Apartment percentage'].median(), inplace=True)
data['Apartment_percentage_live_apt'] = data['Apartment percentage'] * data['Live_in_apartment']
data['Apartment_percentage_not_live_apt'] = data['Apartment percentage'] * (1 - data['Live_in_apartment'])
data['Entropy_live_apt'] = data['entropy'] * data['Live_in_apartment']
data['Entropy_not_live_apt'] = data['entropy'] * (1 - data['Live_in_apartment'])
data['Median_year'].fillna(data['Median_year'].median(), inplace=True)
data['Median_building_age'] = 2023 - data['Median_year']
data['Distance_to_closest_subway_station_log'] = np.log(data['Distance_to_closest_subway_station'])
data['Distance_to_closest_bus_station_log'] = np.log(data['Distance_to_closest_bus_station'])
data['Distance_to_scotiabank_tower_sqrt'] = np.sqrt(data['Distance_to_scotiabank_tower'])
data['Distance_to_scotiabank_tower_squared'] = ((data['Distance_to_scotiabank_tower'])/100) ** 2
data['Distance_to_scotiabank_tower_log'] = np.log(data['Distance_to_scotiabank_tower'])
data['Log_popn_density'] = np.log(data['Popn_density'])
data['Log_entropy'] = np.log(data['entropy'])
data['logsum_transit_smaller'] = data['logsum_transit'] / 10
data = data.rename({'Q14_12': 'Age'}, axis=1)
data['Walk_mode_share'].replace(0, 1e-5, inplace=True)
data['distance_highway_exit'] /= 1000     # Convert to km
data['Children_middle_elementary_school'] = data['Children_in_elementary'] + data['Children_in_middle_school']

In [74]:
exog = data[['Distance_to_closest_subway_station_log', 'Length_of_cycle_lanes_within_radius', 'Log_income', 'pc1', 'Log_popn_density',  'Apartment_percentage_live_apt',
            'Apartment_percentage_not_live_apt', 'Median_building_age', 'Distance_to_scotiabank_tower_log', 'Log_entropy', 'Distance_to_closest_bus_station_log',
            'num_pois_within_radius', 'Children_middle_elementary_school', 'Children_in_high_school',
             'Number_unemployed', 'Licensed_adults', 'Number_telework', 'Own_home', 'Shared_apartment', 'Live_in_apartment',
             'Number_no_fixed_location', 'distance_highway_exit', 'logsum_transit']]
exog_inflation = data[['Distance_to_closest_subway_station_log', 'Length_of_cycle_lanes_within_radius', 'Log_income',
                       'pc1', 'Log_popn_density', 'Apartment_percentage_live_apt', 'Apartment_percentage_not_live_apt', 'Median_building_age', 'Distance_to_scotiabank_tower_log',
                       'Log_entropy', 'Distance_to_closest_bus_station_log', 'num_pois_within_radius', 'Children_middle_elementary_school', 'Children_in_high_school', 'Number_unemployed', 'Licensed_adults',
                       'Number_telework', 'Own_home', 'Shared_apartment', 'Live_in_apartment', 'Number_no_fixed_location',
                       'distance_highway_exit', 'logsum_transit']]

In [75]:
len_exog = len(exog.columns)
len_exog_inflation = len(exog_inflation.columns)

In [76]:
const = params[-1]
const

0.11471026613770796

In [77]:
t1 = params[len_exog]
t1

2.5053124031660996

In [78]:
t2 = params[len_exog + 1]
t2

4.1725456421093

In [79]:
exog_params = params[:len_exog]
exog_params

array([ 0.09022046, -0.22945712,  0.18561338, -0.02144322, -0.19075573,
        0.13801067, -0.13445517, -0.00155763,  0.05360587, -0.02883361,
        0.0211491 , -0.00488325,  0.09791851, -0.06890827, -0.01898759,
        0.92827634, -0.26735259,  0.28348838,  0.05222048, -0.42886745,
        0.19023709, -0.03515157, -0.01199142])

In [80]:
exog_inflation_params = params[len_exog + 2: len_exog + 2 + len_exog_inflation]
exog_inflation_params

array([ 4.64432338e-02, -2.05873546e-01,  1.70906059e-01,  4.44942434e-02,
       -2.37481579e-01, -8.66988566e-02, -3.03859552e-01, -3.36480772e-03,
        1.23552538e-01,  1.27813838e-01, -2.74777554e-03,  1.07876003e-04,
        2.56495241e-01,  8.69150442e-03,  7.08831414e-02,  9.84334126e-01,
       -2.24925889e-01,  4.10854193e-01, -4.40600704e-01, -2.19773138e-01,
        8.60494572e-02, -3.00552689e-02, -3.55829514e-02])

# Ordered Model Marginal Effects

In [81]:
exog.shape

(2875, 23)

In [82]:
exog_params.shape

(23,)

In [83]:
# tmp = pd.Series(-stats.norm.pdf(t1 - np.dot(exog, exog_params)))
tmp = t1 - pd.Series(np.dot(exog, exog_params))
tmp = tmp.astype('float64')

# Convert params to float
exog_params = exog_params.astype('float64')
product_array1 = [-stats.norm.pdf(tmp) * param for param in exog_params]
product_array1

[array([-0.00851818, -0.01651546, -0.0006668 , ..., -0.03403309,
        -0.03516611, -0.03283426]),
 array([0.02166422, 0.04200366, 0.00169586, ..., 0.08655613, 0.08943774,
        0.08350716]),
 array([-0.01752471, -0.03397777, -0.00137182, ..., -0.07001733,
        -0.07234834, -0.06755095]),
 array([0.00202456, 0.00392532, 0.00015848, ..., 0.00808884, 0.00835813,
        0.00780391]),
 array([0.01801022, 0.03491912, 0.00140983, ..., 0.07195714, 0.07435272,
        0.06942242]),
 array([-0.01303029, -0.02526378, -0.00102   , ..., -0.05206057,
        -0.05379376, -0.05022672]),
 array([0.0126946 , 0.02461292, 0.00099372, ..., 0.05071936, 0.0524079 ,
        0.04893276]),
 array([1.47064024e-04, 2.85135011e-04, 1.15120633e-05, ...,
        5.87572117e-04, 6.07133456e-04, 5.66874716e-04]),
 array([-0.0050612 , -0.00981291, -0.00039619, ..., -0.02022128,
        -0.02089448, -0.01950898]),
 array([0.00272233, 0.00527819, 0.0002131 , ..., 0.01087665, 0.01123876,
        0.01049352]),
 a

In [84]:
# d0 is the mean of each array in product_array transformed into a series named d0
d1 = pd.Series([np.mean(array) for array in product_array1])
d1

0    -0.019884
1     0.050572
2    -0.040909
3     0.004726
4     0.042042
5    -0.030417
6     0.029634
7     0.000343
8    -0.011815
9     0.006355
10   -0.004661
11    0.001076
12   -0.021581
13    0.015187
14    0.004185
15   -0.204589
16    0.058924
17   -0.062480
18   -0.011509
19    0.094521
20   -0.041928
21    0.007747
22    0.002643
dtype: float64

In [85]:
# tmp = pd.Series(-stats.norm.pdf(t1 - np.dot(exog, exog_params)))
tmp = t1 - pd.Series(np.dot(exog, exog_params))
tmp = tmp.astype('float64')

# Convert params to float
exog_params = exog_params.astype('float64')
product_array1 = [-stats.norm.pdf(tmp) * param for param in exog_params]
product_array1

[array([-0.00851818, -0.01651546, -0.0006668 , ..., -0.03403309,
        -0.03516611, -0.03283426]),
 array([0.02166422, 0.04200366, 0.00169586, ..., 0.08655613, 0.08943774,
        0.08350716]),
 array([-0.01752471, -0.03397777, -0.00137182, ..., -0.07001733,
        -0.07234834, -0.06755095]),
 array([0.00202456, 0.00392532, 0.00015848, ..., 0.00808884, 0.00835813,
        0.00780391]),
 array([0.01801022, 0.03491912, 0.00140983, ..., 0.07195714, 0.07435272,
        0.06942242]),
 array([-0.01303029, -0.02526378, -0.00102   , ..., -0.05206057,
        -0.05379376, -0.05022672]),
 array([0.0126946 , 0.02461292, 0.00099372, ..., 0.05071936, 0.0524079 ,
        0.04893276]),
 array([1.47064024e-04, 2.85135011e-04, 1.15120633e-05, ...,
        5.87572117e-04, 6.07133456e-04, 5.66874716e-04]),
 array([-0.0050612 , -0.00981291, -0.00039619, ..., -0.02022128,
        -0.02089448, -0.01950898]),
 array([0.00272233, 0.00527819, 0.0002131 , ..., 0.01087665, 0.01123876,
        0.01049352]),
 a

In [86]:
tmp2 = t2 - pd.Series(np.dot(exog, exog_params))
tmp2 = tmp2.astype('float64')

product_array2 = [(stats.norm.pdf(tmp) - stats.norm.pdf(tmp2)) * param for param in exog_params]
product_array2

[array([0.00839301, 0.01600201, 0.0006653 , ..., 0.01922185, 0.02261696,
        0.02883113]),
 array([-0.02134589, -0.04069781, -0.00169205, ..., -0.04888681,
        -0.05752157, -0.07332602]),
 array([0.0172672 , 0.03292144, 0.00136874, ..., 0.03954572, 0.04653058,
        0.05931518]),
 array([-0.00199482, -0.00380329, -0.00015813, ..., -0.00456857,
        -0.0053755 , -0.00685246]),
 array([-0.01774558, -0.03383351, -0.00140666, ..., -0.04064132,
        -0.04781969, -0.06095848]),
 array([0.01283883, 0.02447835, 0.00101771, ..., 0.02940376, 0.03459727,
        0.04410311]),
 array([-0.01250807, -0.02384773, -0.00099149, ..., -0.02864624,
        -0.03370596, -0.0429669 ]),
 array([-1.44903069e-04, -2.76270433e-04, -1.14862110e-05, ...,
        -3.31860137e-04, -3.90475778e-04, -4.97761680e-04]),
 array([0.00498684, 0.00950784, 0.0003953 , ..., 0.01142096, 0.01343821,
        0.01713046]),
 array([-0.00268233, -0.00511409, -0.00021262, ..., -0.00614312,
        -0.00722817, -0.00

In [87]:
d2 = pd.Series([np.mean(array) for array in product_array2])
d2

0     0.014593
1    -0.037115
2     0.030023
3    -0.003468
4    -0.030855
5     0.022323
6    -0.021748
7    -0.000252
8     0.008671
9    -0.004664
10    0.003421
11   -0.000790
12    0.015838
13   -0.011146
14   -0.003071
15    0.150150
16   -0.043245
17    0.045855
18    0.008447
19   -0.069370
20    0.030771
21   -0.005686
22   -0.001940
dtype: float64

In [88]:
# Create a dataframe from the two series, with the column names as 'delta1' and 'delta2'
df = pd.DataFrame({'delta1': d1, 'delta2': d2})
df

,delta1,delta2
0,-0.019884,0.014593
1,0.050572,-0.037115
2,-0.040909,0.030023
3,0.004726,-0.003468
4,0.042042,-0.030855
5,-0.030417,0.022323
6,0.029634,-0.021748
7,0.000343,-0.000252
8,-0.011815,0.008671
9,0.006355,-0.004664


In [89]:
# Concatenate the exog parameters and the dataframe, naming the new column 'params'
exog_params_df = pd.Series(exog_params)
exog_params_df = exog_params_df.rename('params')

df = pd.concat([exog_params_df, df], axis=1)
df

,params,delta1,delta2
0,0.090220,-0.019884,0.014593
1,-0.229457,0.050572,-0.037115
2,0.185613,-0.040909,0.030023
3,-0.021443,0.004726,-0.003468
4,-0.190756,0.042042,-0.030855
5,0.138011,-0.030417,0.022323
6,-0.134455,0.029634,-0.021748
7,-0.001558,0.000343,-0.000252
8,0.053606,-0.011815,0.008671
9,-0.028834,0.006355,-0.004664


In [90]:
# Make the index of the dataframe the names of the exog parameters
df.index = exog.columns
df

,params,delta1,delta2
Distance_to_closest_subway_station_log,0.090220,-0.019884,0.014593
Length_of_cycle_lanes_within_radius,-0.229457,0.050572,-0.037115
Log_income,0.185613,-0.040909,0.030023
pc1,-0.021443,0.004726,-0.003468
Log_popn_density,-0.190756,0.042042,-0.030855
Apartment_percentage_live_apt,0.138011,-0.030417,0.022323
Apartment_percentage_not_live_apt,-0.134455,0.029634,-0.021748
Median_building_age,-0.001558,0.000343,-0.000252
Distance_to_scotiabank_tower_log,0.053606,-0.011815,0.008671
Log_entropy,-0.028834,0.006355,-0.004664


# Binary Model

In [91]:
# Add a constant to exog_inflation
exog_inflation = sm.add_constant(exog_inflation)
exog_inflation

,const,Distance_to_closest_subway_station_log,Length_of_cycle_lanes_within_radius,Log_income,pc1,Log_popn_density,Apartment_percentage_live_apt,Apartment_percentage_not_live_apt,Median_building_age,Distance_to_scotiabank_tower_log,...,Children_in_high_school,Number_unemployed,Licensed_adults,Number_telework,Own_home,Shared_apartment,Live_in_apartment,Number_no_fixed_location,distance_highway_exit,logsum_transit
0,1.0,-0.609325,1.066817,10.308953,-3.025802,10.704266,0.933333,0.000000,37.5,1.357022,...,0,0,2,0,0,1,1,0,0.458,19.702638
1,1.0,0.654484,0.593877,11.156251,-0.726765,11.021252,0.984848,0.000000,10.0,1.999538,...,0,1,2,0,0,0,1,0,0.060,21.062785
2,1.0,-1.872861,0.883507,10.308953,-2.986928,10.490960,1.000000,0.000000,52.5,-0.068533,...,0,1,1,0,0,0,1,0,0.458,21.548520
3,1.0,-0.647650,1.022463,12.206073,-3.210088,9.299486,0.000000,0.660377,73.0,0.835949,...,0,1,2,0,1,0,0,0,0.458,20.924363
4,1.0,-0.609325,1.066817,10.308953,-3.025802,11.021252,0.984848,0.000000,10.0,1.357022,...,0,0,0,0,0,1,1,0,0.458,19.702638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890,1.0,0.817512,0.776834,10.308953,-3.282090,9.119617,0.000000,0.687500,73.0,1.486582,...,1,1,1,1,0,0,0,0,1.140,23.221603
2891,1.0,-0.309784,0.981833,12.072541,-2.701897,7.819757,0.000000,0.474227,15.0,1.559232,...,0,1,2,0,1,0,0,0,1.140,22.205538
2892,1.0,0.031171,1.031468,12.206073,-2.363758,8.743245,0.000000,0.564885,20.0,1.767601,...,0,0,3,1,1,0,0,0,1.583,18.164385
2893,1.0,3.562760,0.752801,12.206073,0.832389,7.901081,0.000000,0.000000,27.5,3.871158,...,0,2,2,0,1,0,0,0,1.058,23.226725


In [92]:
exog_inflation_params

array([ 4.64432338e-02, -2.05873546e-01,  1.70906059e-01,  4.44942434e-02,
       -2.37481579e-01, -8.66988566e-02, -3.03859552e-01, -3.36480772e-03,
        1.23552538e-01,  1.27813838e-01, -2.74777554e-03,  1.07876003e-04,
        2.56495241e-01,  8.69150442e-03,  7.08831414e-02,  9.84334126e-01,
       -2.24925889e-01,  4.10854193e-01, -4.40600704e-01, -2.19773138e-01,
        8.60494572e-02, -3.00552689e-02, -3.55829514e-02])

In [93]:
# Append const at the start of the exog_inflation_params
exog_inflation_params = np.append(const, exog_inflation_params)
exog_inflation_params

array([ 1.14710266e-01,  4.64432338e-02, -2.05873546e-01,  1.70906059e-01,
        4.44942434e-02, -2.37481579e-01, -8.66988566e-02, -3.03859552e-01,
       -3.36480772e-03,  1.23552538e-01,  1.27813838e-01, -2.74777554e-03,
        1.07876003e-04,  2.56495241e-01,  8.69150442e-03,  7.08831414e-02,
        9.84334126e-01, -2.24925889e-01,  4.10854193e-01, -4.40600704e-01,
       -2.19773138e-01,  8.60494572e-02, -3.00552689e-02, -3.55829514e-02])

In [94]:
tmp3 = np.dot(exog_inflation, exog_inflation_params)
tmp3 = pd.Series(tmp3).astype('float64')

product_array_inflation = [stats.norm.pdf(tmp3) * param for param in exog_inflation_params]
product_array_inflation

[array([0.03871368, 0.04232323, 0.0208604 , ..., 0.0043945 , 0.0029626 ,
        0.01546713]),
 array([0.01567417, 0.01713559, 0.00844584, ..., 0.00177922, 0.00119948,
        0.00626224]),
 array([-0.06948047, -0.07595862, -0.0374387 , ..., -0.00788693,
        -0.00531706, -0.02775926]),
 array([0.05767926, 0.0630571 , 0.03107976, ..., 0.00654734, 0.00441396,
        0.02304437]),
 array([0.01501641, 0.01641649, 0.00809141, ..., 0.00170456, 0.00114915,
        0.00599945]),
 array([-0.0801479 , -0.08762064, -0.04318672, ..., -0.00909782,
        -0.0061334 , -0.03202118]),
 array([-0.02926008, -0.03198821, -0.01576644, ..., -0.0033214 ,
        -0.00223916, -0.01169017]),
 array([-0.10254986, -0.1121113 , -0.05525774, ..., -0.01164073,
        -0.00784773, -0.04097135]),
 array([-1.13559232e-03, -1.24147150e-03, -6.11900068e-04, ...,
        -1.28904376e-04, -8.69023152e-05, -4.53698823e-04]),
 array([0.04169787, 0.04558565, 0.02246839, ..., 0.00473325, 0.00319097,
        0.01665939

In [95]:
delta_inflation = pd.Series([np.mean(array) for array in product_array_inflation])
delta_inflation

0     0.022645
1     0.009169
2    -0.040642
3     0.033739
4     0.008784
5    -0.046882
6    -0.017116
7    -0.059986
8    -0.000664
9     0.024391
10    0.025232
11   -0.000542
12    0.000021
13    0.050636
14    0.001716
15    0.013993
16    0.194321
17   -0.044404
18    0.081108
19   -0.086981
20   -0.043386
21    0.016987
22   -0.005933
23   -0.007025
dtype: float64

In [96]:
# Concatenate the exog_inflation_params and delta_inflation
exog_inflation_params_df = pd.Series(exog_inflation_params)
exog_inflation_params_df = exog_inflation_params_df.rename('Inflation params')

delta_inflation_df = pd.DataFrame({'delta_inflation': delta_inflation})
delta_inflation_df

df2 = pd.concat([exog_inflation_params_df, delta_inflation_df], axis=1)
df2

,Inflation params,delta_inflation
0,0.114710,0.022645
1,0.046443,0.009169
2,-0.205874,-0.040642
3,0.170906,0.033739
4,0.044494,0.008784
5,-0.237482,-0.046882
6,-0.086699,-0.017116
7,-0.303860,-0.059986
8,-0.003365,-0.000664
9,0.123553,0.024391


In [97]:
# Make the index of the dataframe the names of the exog_inflation parameters
df2.index = exog_inflation.columns
df2

,Inflation params,delta_inflation
const,0.114710,0.022645
Distance_to_closest_subway_station_log,0.046443,0.009169
Length_of_cycle_lanes_within_radius,-0.205874,-0.040642
Log_income,0.170906,0.033739
pc1,0.044494,0.008784
Log_popn_density,-0.237482,-0.046882
Apartment_percentage_live_apt,-0.086699,-0.017116
Apartment_percentage_not_live_apt,-0.303860,-0.059986
Median_building_age,-0.003365,-0.000664
Distance_to_scotiabank_tower_log,0.123553,0.024391


In [98]:
# Merge the two dataframes on index
df3 = pd.merge(df2, df, left_index=True, right_index=True)
df3

,Inflation params,delta_inflation,params,delta1,delta2
Distance_to_closest_subway_station_log,0.046443,0.009169,0.090220,-0.019884,0.014593
Length_of_cycle_lanes_within_radius,-0.205874,-0.040642,-0.229457,0.050572,-0.037115
Log_income,0.170906,0.033739,0.185613,-0.040909,0.030023
pc1,0.044494,0.008784,-0.021443,0.004726,-0.003468
Log_popn_density,-0.237482,-0.046882,-0.190756,0.042042,-0.030855
Apartment_percentage_live_apt,-0.086699,-0.017116,0.138011,-0.030417,0.022323
Apartment_percentage_not_live_apt,-0.303860,-0.059986,-0.134455,0.029634,-0.021748
Median_building_age,-0.003365,-0.000664,-0.001558,0.000343,-0.000252
Distance_to_scotiabank_tower_log,0.123553,0.024391,0.053606,-0.011815,0.008671
Log_entropy,0.127814,0.025232,-0.028834,0.006355,-0.004664


In [99]:
# Save to excel
df3.to_excel(f'marginal_effects_{scenario_name[-70:]}.xlsx')

In [100]:
# Normalize the marginal effects by the standard deviation of the exog variables
df4 = df3.copy()
df4

# Get the standard deviation of the exog variables
std = exog.std()
std

Distance_to_closest_subway_station_log     1.660651
Length_of_cycle_lanes_within_radius        0.298504
Log_income                                 0.595209
pc1                                        4.234252
Log_popn_density                           1.222450
Apartment_percentage_live_apt              0.452482
Apartment_percentage_not_live_apt          0.246327
Median_building_age                       23.116589
Distance_to_scotiabank_tower_log           1.152184
Log_entropy                                0.455383
Distance_to_closest_bus_station_log        0.773506
num_pois_within_radius                    13.629258
Children_middle_elementary_school          0.573681
Children_in_high_school                    0.302731
Number_unemployed                          0.760512
Licensed_adults                            0.851463
Number_telework                            0.551332
Own_home                                   0.471075
Shared_apartment                           0.188474
Live_in_apar

In [101]:
# Normalize the marginal effects by the standard deviation of the exog variables
df4['delta1_normalized'] = df4['delta1'] / std
df4['delta2_normalized'] = df4['delta2'] / std
df4['delta_inflation_normalized'] = df4['delta_inflation'] / std
df4

,Inflation params,delta_inflation,params,delta1,delta2,delta1_normalized,delta2_normalized,delta_inflation_normalized
Distance_to_closest_subway_station_log,0.046443,0.009169,0.090220,-0.019884,0.014593,-0.011974,0.008788,0.005521
Length_of_cycle_lanes_within_radius,-0.205874,-0.040642,-0.229457,0.050572,-0.037115,0.169417,-0.124337,-0.136153
Log_income,0.170906,0.033739,0.185613,-0.040909,0.030023,-0.068730,0.050442,0.056685
pc1,0.044494,0.008784,-0.021443,0.004726,-0.003468,0.001116,-0.000819,0.002074
Log_popn_density,-0.237482,-0.046882,-0.190756,0.042042,-0.030855,0.034392,-0.025240,-0.038351
Apartment_percentage_live_apt,-0.086699,-0.017116,0.138011,-0.030417,0.022323,-0.067223,0.049336,-0.037826
Apartment_percentage_not_live_apt,-0.303860,-0.059986,-0.134455,0.029634,-0.021748,0.120301,-0.088290,-0.243522
Median_building_age,-0.003365,-0.000664,-0.001558,0.000343,-0.000252,0.000015,-0.000011,-0.000029
Distance_to_scotiabank_tower_log,0.123553,0.024391,0.053606,-0.011815,0.008671,-0.010254,0.007526,0.021169
Log_entropy,0.127814,0.025232,-0.028834,0.006355,-0.004664,0.013955,-0.010242,0.055409


In [102]:
# Save to excel
df4.to_excel(f'marginal_effects_normalized_{scenario_name[-70:]}_2.xlsx')